In [30]:
# Giving a seed value.Apparently you may use different seed values at each stage
seed_value= 0

# 4 steps that are neeeded to be done for the code to be reproducible.

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

# 5. Configure a new global `tensorflow` session
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# Check GPU availability
print("GPU Available:", tf.config.list_physical_devices("GPU"))

# Check CPU availability
print("CPU Available:", tf.config.list_physical_devices("CPU"))

#Import all the necessary libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler 
from keras.models import Sequential #importing Sequential Model.It uses tensorflow backend and you need to have tensorflow setup. 
from keras.layers import Dense #Importing default dense layer
from sklearn.utils import shuffle
from keras.regularizers import l1, l2
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
import time
from matplotlib import pyplot


GPU Available: []
CPU Available: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


### Accessing all the data

In [31]:
%store -r appended_data
%store -r new_data

df=appended_data.copy()

x_axis=df.iloc[:,5:114]
x_axis.columns

Index(['SPEED', 'STOPPER', 'TUNDISHWEIGHT', 'TUNDISHTEMPERATURE',
       'BROADFACE1WATERFLOW', 'BROADFACE2WATERFLOW', 'NARROWFACE1WATERFLOW',
       'NARROWFACE2WATERFLOW', 'MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTA',
       'MOLDBROADFACE2INLETOUTLETWATERTEMP.DELTA',
       ...
       'DAYSAFTERSPRAYCLEANING', 'PER_ALC', 'PER_ALN', 'PER_NBC', 'PER_VN',
       'PER_TIN', 'PER_TIC', 'PER_MNS', 'PER_VC', 'PER_NBN'],
      dtype='object', length=109)

### Calculating mean

In [32]:
def variable_constant(variable):
    for row in range (0,len(new_data)): #will walk through all heat numbers
        mean_value=np.mean(new_data[row][1][variable])
        mean_array.append(mean_value)
    meandata[variable+'MEAN']=mean_array
    
const_selected=[]
meandata = pd.DataFrame()
for variable_index in range(0,len(x_axis.columns),1):
    mean_array=[]
    current_variable=x_axis.columns[variable_index]
    variable_constant(current_variable)
    const_selected.append(x_axis.columns[variable_index])
    
op_data=[]
for row in range (0,len(new_data)):
    op_data.append(new_data[row][3])
meandata["OUTPUT"]=op_data
meandata

C:\Users\subhi\AppData\Local\Temp\ipykernel_17836\376425405.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meandata[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_17836\376425405.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  meandata[variable+'MEAN']=mean_array
C:\Users\subhi\AppData\Local\Temp\ipykernel_17836\376425405.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all co

,SPEEDMEAN,STOPPERMEAN,TUNDISHWEIGHTMEAN,TUNDISHTEMPERATUREMEAN,BROADFACE1WATERFLOWMEAN,BROADFACE2WATERFLOWMEAN,NARROWFACE1WATERFLOWMEAN,NARROWFACE2WATERFLOWMEAN,MOLDBROADFACE1INLETOUTLETWATERTEMP.DELTAMEAN,MOLDBROADFACE2INLETOUTLETWATERTEMP.DELTAMEAN,...,PER_ALCMEAN,PER_ALNMEAN,PER_NBCMEAN,PER_VNMEAN,PER_TINMEAN,PER_TICMEAN,PER_MNSMEAN,PER_VCMEAN,PER_NBNMEAN,OUTPUT
0,1.120270,50.758621,23.854885,1554.574713,3826.942529,3820.425287,461.735632,463.959770,4.000000,4.000000,...,0.002079,0.000137,0.001267,0.000024,0.000043,0.000651,0.002926,0.000371,0.000083,1
1,1.114253,56.146465,23.548081,1548.237374,3823.025253,3823.176768,461.954545,467.464646,4.000000,4.000000,...,0.002678,0.000180,0.001382,0.000026,0.000053,0.000788,0.003059,0.000387,0.000093,1
2,1.198736,61.086207,24.320115,1551.000000,3821.913793,3821.626437,461.534483,468.091954,4.000000,4.000000,...,0.002037,0.000163,0.001260,0.000028,0.000048,0.000595,0.003990,0.000350,0.000101,0
3,1.198899,65.455696,24.275696,1547.107595,3826.860759,3826.151899,461.892405,466.177215,4.000000,4.000000,...,0.001939,0.000144,0.001344,0.000026,0.000048,0.000651,0.002660,0.000350,0.000100,0
4,1.141729,66.005650,24.563785,1553.090395,3823.536723,3823.525424,462.028249,465.830508,4.000000,4.000000,...,0.002113,0.000151,0.001285,0.000027,0.000043,0.000607,0.003725,0.000378,0.000092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586,0.983690,71.732143,23.541369,1545.142857,4000.803571,4000.636905,452.720238,451.625000,7.994048,7.535714,...,0.006482,0.000170,0.000421,0.000013,0.000010,0.000366,0.003682,0.000513,0.000011,0
587,0.795487,69.920635,27.779471,1555.682540,3958.724868,3966.460317,443.820106,451.359788,8.052910,7.941799,...,0.006992,0.000253,0.000426,0.000239,0.000004,0.000122,0.004050,0.006597,0.000015,1
588,0.911294,73.304124,22.814948,1537.123711,4052.396907,4024.128866,452.283505,456.850515,7.407216,7.273196,...,0.009100,0.000260,0.005642,0.000026,0.000081,0.002839,0.018383,0.000910,0.000161,1
589,0.918199,67.461165,24.742087,1543.466019,4030.233010,4027.310680,452.150485,451.485437,7.121359,7.000000,...,0.008918,0.000255,0.005460,0.000025,0.000076,0.002657,0.016292,0.000874,0.000156,1


### Training function

In [47]:
def training_data(new_features,n_batch):
    # Creating a new data
    new_data = meandata.loc[:,new_features] # Taking the features given by forward selection algorithm.
    new_data = pd.concat([new_data, meandata['OUTPUT']], axis=1)  # Add the 'OUTPUT' column
    new_data = shuffle(new_data, random_state=42) #Shuffling the data along with the output column.
    
    #Taking all independent variables from shuffled new_data in X
    X=new_data.iloc[:,:-1]

    # Splitting the shuffled new_data into train and test
    new_total_length = X.shape[0]
    new_train_size = int(X.shape[0] * 0.8)
    new_test_size = X.shape[0] - new_train_size

    new_train_X = X.iloc[:new_train_size, :].to_numpy().astype(np.float32)  # 2D array
    new_train_Y = new_data['OUTPUT'][:new_train_size].to_numpy().astype(np.float32)  # 1D array

    new_test_X = X.iloc[new_train_size:, :].to_numpy().astype(np.float32)  # 2D array
    new_test_Y = new_data['OUTPUT'][new_train_size:].to_numpy().astype(np.float32)  # 1D array
    
    new_train_Y=np.reshape(new_train_Y,(new_train_Y.shape[0],1)) # 2D array
    new_test_Y=np.reshape(new_test_Y,(new_test_Y.shape[0],1)) # 2D array}

    # Scaling the data
    scaler = RobustScaler()
    new_train_X = scaler.fit_transform(new_train_X)
    new_test_X = scaler.transform(new_test_X)

    # Create a new model with the updated feature set (Random Forest Classifier)
    new_model=Sequential() 
    new_model.add(Dense(50,input_dim=len(new_features),activation='relu',kernel_regularizer=l2(0.01),kernel_initializer='he_uniform')) # one hidden layer
    new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer

    #4 Compile the new model
    new_model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.01,momentum=0.9),metrics=['accuracy'])

    #5 Train the new model
    history=new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=n_batch,validation_data=(new_test_X,new_test_Y))
    
    #6 Evaluate the new model
    testPredict_probs_new=new_model.predict(new_test_X)
    threshold = 0.5  # Adjust this threshold as needed
    y_new_pred = (testPredict_probs_new > threshold).astype(int)
    new_accuracy = accuracy_score(new_test_Y,y_new_pred)

    return new_accuracy,history

### Forward Feature Selection function

In [48]:
total_features_list=meandata.columns.tolist()

In [49]:
def forward_selection(n_batch):
    # Start the timer
    start_time = time.time()
    
    selected_features=[] # starting with a empty feature set
    selected_features_accuracy=[] # for storing accuracy of selected feature
    condition=False
    count=0 #to know how many times my loop runs
    accuracy=-np.inf #starting with a very low accuracy
    
    while (condition==False):
        remaining_features=[]# Remaining features will change at each iteration
        
        for feature in total_features_list:
            if feature not in selected_features:
                remaining_features.append(feature) 
                
        temp_accuracy_store=[]
        temp_feature_array=[]
        last_feature=remaining_features[-1]
        temp_model_history=[]
        
        for feature in remaining_features:
            # Add one feature to the feature set
            new_features = selected_features + [feature]
            print(feature)
            new_accuracy,new_model_history=training_data(new_features,n_batch)
            temp_accuracy_store.append(new_accuracy)
            temp_feature_array.append(feature)
            temp_model_history.append(new_model_history)
           
        maximum_accuracy=max(temp_accuracy_store)
            
        # Check for convergene. Stopping when no improvement.
        if maximum_accuracy<=accuracy:
            condition=True
            break       

        # Update the baseline feature set if the accuracy score improved
        else:
            max_idx=np.argmax(temp_accuracy_store)
            accuracy=maximum_accuracy #updating the accuracy
            selected_features.append(temp_feature_array[max_idx]) #updating the selected features
            selected_features_accuracy.append(accuracy)
            history=temp_model_history[max_idx]
            count=count+1
            # plot learning curves only for the feature set selected each time
            pyplot.plot(history.history['accuracy'], label='train')
            pyplot.plot(history.history['val_accuracy'], label='test')
            pyplot.title('batch=' + str(n_batch) + ',Accuracy=' + str(round(accuracy * 100, 4)) + '\nFeature=' + temp_feature_array[max_idx], pad=-40, fontsize='10')
            pyplot.legend()
            pyplot.savefig(f'learning_curves_iteration{count}.png')
            pyplot.clf()
            
    # Stop the timer and calculate the elapsed time
    end_time = time.time()
    training_time = end_time - start_time       
    return selected_features, selected_features_accuracy, training_time,temp_accuracy_store,temp_model_history,history

In [50]:
selected_features,selected_features_accuracy,training_time,temp_accuracy_store,temp_model_history,history=forward_selection(472)

SPEEDMEAN
Epoch 1/150


C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 1s 580ms/step - loss: 1.7869 - accuracy: 0.5869 - val_loss: 1.7942 - val_accuracy: 0.6050
Epoch 2/150
1/1 [==============================] - 0s 63ms/step - loss: 1.7862 - accuracy: 0.5869 - val_loss: 1.7933 - val_accuracy: 0.6050
Epoch 3/150
1/1 [==============================] - 0s 47ms/step - loss: 1.7849 - accuracy: 0.5869 - val_loss: 1.7919 - val_accuracy: 0.6050
Epoch 4/150
1/1 [==============================] - 0s 47ms/step - loss: 1.7831 - accuracy: 0.5869 - val_loss: 1.7903 - val_accuracy: 0.6050
Epoch 5/150
1/1 [==============================] - 0s 47ms/step - loss: 1.7809 - accuracy: 0.5869 - val_loss: 1.7884 - val_accuracy: 0.6050
Epoch 6/150
1/1 [==============================] - 0s 47ms/step - loss: 1.7784 - accuracy: 0.5869 - val_loss: 1.7864 - val_accuracy: 0.6050
Epoch 7/150
1/1 [==============================] - 0s 31ms/step - loss: 1.7757 - accuracy: 0.5890 - val_loss: 1.7843 - val_accuracy: 0.6050
Epoch 8/150
1/1 [==============

1/1 [==============================] - 0s 47ms/step - loss: 1.5813 - accuracy: 0.5975 - val_loss: 1.5947 - val_accuracy: 0.5546
Epoch 60/150
1/1 [==============================] - 0s 47ms/step - loss: 1.5776 - accuracy: 0.5975 - val_loss: 1.5910 - val_accuracy: 0.5546
Epoch 61/150
1/1 [==============================] - 0s 48ms/step - loss: 1.5739 - accuracy: 0.5975 - val_loss: 1.5874 - val_accuracy: 0.5462
Epoch 62/150
1/1 [==============================] - 0s 46ms/step - loss: 1.5702 - accuracy: 0.5975 - val_loss: 1.5838 - val_accuracy: 0.5462
Epoch 63/150
1/1 [==============================] - 0s 31ms/step - loss: 1.5666 - accuracy: 0.5975 - val_loss: 1.5802 - val_accuracy: 0.5462
Epoch 64/150
1/1 [==============================] - 0s 32ms/step - loss: 1.5629 - accuracy: 0.5975 - val_loss: 1.5766 - val_accuracy: 0.5462
Epoch 65/150
1/1 [==============================] - 0s 31ms/step - loss: 1.5593 - accuracy: 0.5975 - val_loss: 1.5730 - val_accuracy: 0.5462
Epoch 66/150
1/1 [========

1/1 [==============================] - 0s 30ms/step - loss: 1.3900 - accuracy: 0.6165 - val_loss: 1.4053 - val_accuracy: 0.5294
Epoch 118/150
1/1 [==============================] - 0s 31ms/step - loss: 1.3871 - accuracy: 0.6165 - val_loss: 1.4024 - val_accuracy: 0.5294
Epoch 119/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3842 - accuracy: 0.6144 - val_loss: 1.3995 - val_accuracy: 0.5294
Epoch 120/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3813 - accuracy: 0.6144 - val_loss: 1.3966 - val_accuracy: 0.5294
Epoch 121/150
1/1 [==============================] - 0s 32ms/step - loss: 1.3784 - accuracy: 0.6144 - val_loss: 1.3938 - val_accuracy: 0.5294
Epoch 122/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3755 - accuracy: 0.6144 - val_loss: 1.3909 - val_accuracy: 0.5294
Epoch 123/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3727 - accuracy: 0.6144 - val_loss: 1.3881 - val_accuracy: 0.5294
Epoch 124/150
1/1 [=

...

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1/1 [==============================] - 1s 507ms/step - loss: 1.9834 - accuracy: 0.4767 - val_loss: 1.9166 - val_accuracy: 0.5462
Epoch 2/150
1/1 [==============================] - 0s 47ms/step - loss: 1.9452 - accuracy: 0.4746 - val_loss: 1.8512 - val_accuracy: 0.5546
Epoch 3/150
1/1 [==============================] - 0s 31ms/step - loss: 1.8762 - accuracy: 0.4831 - val_loss: 1.7684 - val_accuracy: 0.5798
Epoch 4/150
1/1 [==============================] - 0s 31ms/step - loss: 1.7884 - accuracy: 0.4746 - val_loss: 1.6908 - val_accuracy: 0.5462
Epoch 5/150
1/1 [==============================] - 0s 47ms/step - loss: 1.7035 - accuracy: 0.4852 - val_loss: 1.6594 - val_accuracy: 0.5126
Epoch 6/150
1/1 [==============================] - 0s 47ms/step - loss: 1.6582 - accuracy: 0.5297 - val_loss: 1.6948 - val_accuracy: 0.5126
Epoch 7/150
1/1 [==============================] - 0s 32ms/step - loss: 1.6724 - accuracy: 0.5254 - val_loss: 1.7562 - val_accuracy: 0.5126
Epoch 8/150
1/1 [==============

1/1 [==============================] - 0s 31ms/step - loss: 1.4821 - accuracy: 0.5360 - val_loss: 1.4863 - val_accuracy: 0.5042
Epoch 60/150
1/1 [==============================] - 0s 32ms/step - loss: 1.4790 - accuracy: 0.5318 - val_loss: 1.4841 - val_accuracy: 0.5042
Epoch 61/150
1/1 [==============================] - 0s 47ms/step - loss: 1.4760 - accuracy: 0.5318 - val_loss: 1.4815 - val_accuracy: 0.5042
Epoch 62/150
1/1 [==============================] - 0s 47ms/step - loss: 1.4730 - accuracy: 0.5318 - val_loss: 1.4786 - val_accuracy: 0.5042
Epoch 63/150
1/1 [==============================] - 0s 31ms/step - loss: 1.4698 - accuracy: 0.5318 - val_loss: 1.4751 - val_accuracy: 0.5042
Epoch 64/150
1/1 [==============================] - 0s 37ms/step - loss: 1.4666 - accuracy: 0.5318 - val_loss: 1.4714 - val_accuracy: 0.5042
Epoch 65/150
1/1 [==============================] - 0s 42ms/step - loss: 1.4633 - accuracy: 0.5318 - val_loss: 1.4675 - val_accuracy: 0.5126
Epoch 66/150
1/1 [========

1/1 [==============================] - 0s 38ms/step - loss: 1.3155 - accuracy: 0.5403 - val_loss: 1.3197 - val_accuracy: 0.5042
Epoch 118/150
1/1 [==============================] - 0s 54ms/step - loss: 1.3129 - accuracy: 0.5403 - val_loss: 1.3172 - val_accuracy: 0.5042
Epoch 119/150
1/1 [==============================] - 0s 39ms/step - loss: 1.3104 - accuracy: 0.5403 - val_loss: 1.3147 - val_accuracy: 0.5042
Epoch 120/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3079 - accuracy: 0.5403 - val_loss: 1.3122 - val_accuracy: 0.5042
Epoch 121/150
1/1 [==============================] - 0s 32ms/step - loss: 1.3054 - accuracy: 0.5403 - val_loss: 1.3097 - val_accuracy: 0.5042
Epoch 122/150
1/1 [==============================] - 0s 31ms/step - loss: 1.3029 - accuracy: 0.5403 - val_loss: 1.3071 - val_accuracy: 0.5042
Epoch 123/150
1/1 [==============================] - 0s 47ms/step - loss: 1.3004 - accuracy: 0.5403 - val_loss: 1.3046 - val_accuracy: 0.5042
Epoch 124/150
1/1 [=

ValueError: cannot reshape array of size 944 into shape (472,1)

In [43]:
selected_features

NameError: name 'selected_features' is not defined

In [17]:
selected_features_accuracy

[0.6722689075630253, 0.7647058823529411, 0.773109243697479, 0.8151260504201681]

In [34]:
# Creating a new data
new_data = meandata.loc[:,'SPEEDMEAN'] # Taking the features given by forward selection algorithm.
new_data = pd.concat([new_data, meandata['OUTPUT']], axis=1)  # Add the 'OUTPUT' column
new_data = shuffle(new_data, random_state=42) #Shuffling the data along with the output column.
new_data

,SPEEDMEAN,OUTPUT
505,0.927653,1
586,0.983690,0
195,0.911936,0
165,1.082177,1
453,1.145806,1
...,...,...
71,0.939836,1
106,0.976015,0
270,1.083643,0
435,1.224015,0


In [35]:
#Taking all independent variables from shuffled new_data in X
X=new_data.iloc[:,:-1]

# Splitting the shuffled new_data into train and test
new_total_length = X.shape[0]
new_train_size = int(X.shape[0] * 0.8)
new_test_size = X.shape[0] - new_train_size

new_train_X = X.iloc[:new_train_size, :].to_numpy().astype(np.float32)  # 2D array
new_train_Y = new_data['OUTPUT'][:new_train_size].to_numpy().astype(np.float32)  # 1D array

new_test_X = X.iloc[new_train_size:, :].to_numpy().astype(np.float32)  # 2D array
new_test_Y = new_data['OUTPUT'][new_train_size:].to_numpy().astype(np.float32)  # 1D array

In [37]:
new_train_X.shape

(472, 1)

In [38]:
new_train_Y.shape

(472,)

In [54]:
# Creating a new data
new_data = meandata.loc[:,['SPEEDMEAN','STOPPERMEAN']] # Taking the features given by forward selection algorithm.
new_data = pd.concat([new_data, meandata['OUTPUT']], axis=1)  # Add the 'OUTPUT' column
new_data = shuffle(new_data, random_state=42) #Shuffling the data along with the output column.

#Taking all independent variables from shuffled new_data in X
X=new_data.iloc[:,:-1]

# Splitting the shuffled new_data into train and test
new_total_length = X.shape[0]
new_train_size = int(X.shape[0] * 0.8)
new_test_size = X.shape[0] - new_train_size

new_train_X = X.iloc[:new_train_size, :].to_numpy().astype(np.float32)  # 2D array
new_train_Y = new_data['OUTPUT'][:new_train_size].to_numpy().astype(np.float32)  # 1D array

new_test_X = X.iloc[new_train_size:, :].to_numpy().astype(np.float32)  # 2D array
new_test_Y = new_data['OUTPUT'][new_train_size:].to_numpy().astype(np.float32)  # 1D array

new_train_Y=np.reshape(new_train_Y,(new_train_Y.shape[0],1)) # 2D array
new_test_Y=np.reshape(new_test_Y,(new_test_Y.shape[0],1)) # 2D array}

# Scaling the data
scaler = RobustScaler()
new_train_X = scaler.fit_transform(new_train_X)
new_test_X = scaler.transform(new_test_X)

# Create a new model with the updated feature set (Random Forest Classifier)
new_model=Sequential() 
new_model.add(Dense(50,input_dim=1,activation='relu',kernel_regularizer=l2(0.01),kernel_initializer='he_uniform')) # one hidden layer
new_model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.01))) #output layer

#4 Compile the new model
new_model.compile(loss='binary_crossentropy',optimizer=SGD(lr=0.01,momentum=0.9),metrics=['accuracy'])

#5 Train the new model
history=new_model.fit(new_train_X,new_train_Y,epochs=150,batch_size=472,validation_data=(new_test_X,new_test_Y))

C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/150


ValueError: in user code:

    File "C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\subhi\anaconda3\envs\tensorflow\lib\site-packages\keras\engine\input_spec.

In [52]:
#6 Evaluate the new model
testPredict_probs_new=new_model.predict(new_test_X)
threshold = 0.5  # Adjust this threshold as needed
y_new_pred = (testPredict_probs_new > threshold).astype(int)
new_accuracy = accuracy_score(new_test_Y,y_new_pred)

new_accuracy,history

4/4 [==============================] - 0s 5ms/step


(0.5378151260504201, <keras.callbacks.History at 0x256741d6fe0>)